In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
import os
from scipy import optimize

In [4]:
thepath = os.path.join(os.getcwd(),'..')
n_singles = pd.read_csv(os.path.join(thepath,'data_mec_optim/marriage-ChooSiow/n_singles.txt'), sep='\t', header = None)
marr = pd.read_csv(os.path.join(thepath,'data_mec_optim/marriage-ChooSiow/marr.txt'), sep='\t', header = None)
navail = pd.read_csv(os.path.join(thepath,'data_mec_optim/marriage-ChooSiow/n_avail.txt'), sep='\t', header = None)

In [5]:
nbCateg = 25

In [6]:
muhat_x0 = n_singles[0].iloc[0:nbCateg]
muhat_0y = n_singles[1].iloc[0:nbCateg]
muhat_xy = marr.iloc[0:nbCateg:,0:nbCateg]

In [7]:
Nh = muhat_xy.values.sum()+muhat_x0.sum()+muhat_0y.sum()

In [8]:
muhat_xy = muhat_xy / Nh 
muhat_x0 = muhat_x0 / Nh 
muhat_0y = muhat_0y / Nh

In [9]:
n_x = muhat_xy.sum(axis = 1)+muhat_x0
m_y = muhat_xy.sum(axis = 0)+muhat_0y

In [10]:
nbX = nbCateg
nbY = nbCateg

xs = np.repeat(range(1,nbX+1),nbY).reshape(nbX,nbY)
ys = np.repeat(range(1,nbY+1),nbX).reshape(nbX,nbY).T

phi1_xy = -((xs-ys)**2).flatten()
phimat = np.column_stack((phi1_xy,np.multiply(phi1_xy,(((xs+ys)/2)**2).flatten()),np.multiply(phi1_xy,(((xs+ys-2)/2)**2).flatten()),np.multiply(phi1_xy,((xs+ys-1)**2).flatten())))

In [11]:
nbK = phimat.shape[1]
phimat_mean = phimat.mean(axis = 0)
phimat_stdev = phimat.std(axis = 0, ddof = 1)
phimat = ((phimat - phimat_mean).T/phimat_stdev[:,None]).T

In [12]:
def ObjFunc(uvlambda):
    u_x = uvlambda[0:nbX]
    v_y = uvlambda[nbX:(nbX+nbY)]
    l = uvlambda[(nbX+nbY):(nbX+nbY+nbK)]
    
    Phi_xy = phimat.dot(l.reshape(nbK,1)).reshape(nbX, nbY)
    mu_xy = np.exp(((Phi_xy - u_x).T-v_y).T/2)
    mu_x0 = np.exp(-u_x)
    mu_0y = np.exp(-v_y)
    
    val = sum(np.multiply(n_x,u_x))+sum(np.multiply(m_y,v_y))-np.sum(np.multiply(muhat_xy.values,Phi_xy), axis = (0,1)) + 2*np.sum(mu_xy, axis =(0,1)) + sum(mu_x0) + sum(mu_0y)
    
    return val

In [13]:
def grad_ObjFunc(uvlambda):
    u_x = uvlambda[0:nbX]
    v_y = uvlambda[nbX:(nbX+nbY)]
    l = uvlambda[(nbX+nbY):(nbX+nbY+nbK)]
    
    Phi_xy = phimat.dot(l.reshape(nbK,1)).reshape(nbX, nbY)
    mu_xy = np.exp(((Phi_xy - u_x).T-v_y).T/2)
    mu_x0 = np.exp(-u_x)
    mu_0y = np.exp(-v_y)
    
    grad_u = n_x - np.sum(mu_xy, axis = 0) - mu_x0
    grad_v = m_y - np.sum(mu_xy, axis = 1) - mu_0y
    grad_lambda = (mu_xy-muhat_xy.values).flatten()[:,None].T.dot(phimat)
    
    grad = np.concatenate((grad_u,grad_v,grad_lambda.flatten()))
    
    return grad

In [14]:
outcome = optimize.minimize(ObjFunc,method = 'CG',jac = grad_ObjFunc, x0 = np.repeat(0,nbX+nbY+nbK))

In [15]:
uvlambdahat =  outcome['x']
lambdahat = uvlambdahat[(nbX+nbY):(nbX+nbY+nbK)]
print(outcome)
print("")
print(ObjFunc(uvlambdahat))
print(lambdahat)

     fun: 7.677074350528977
     jac: array([ 7.44467533e-06,  7.07844255e-06, -1.04446028e-06,  7.60661206e-06,
       -5.33720810e-07,  8.92370675e-07, -8.86179675e-07, -4.26218021e-06,
        9.85511851e-07,  5.53458302e-07,  1.15237789e-06,  1.02375752e-06,
        2.20435318e-08,  5.98853927e-07,  5.51234130e-07, -6.33418436e-07,
        1.75557297e-06,  2.02871072e-06,  3.13680060e-06,  2.62580017e-06,
        2.11404188e-06,  1.65848052e-06,  3.26267816e-06,  8.52096064e-07,
        2.89268297e-07,  5.69001608e-06, -4.84642223e-06, -3.75286073e-06,
        2.41258263e-06,  1.59780895e-06, -2.70885423e-07, -1.91356398e-06,
       -1.26096491e-06, -1.50637299e-06, -6.93378788e-06, -1.72848228e-06,
       -1.51205532e-06, -1.11931175e-06, -9.52343656e-07, -8.64416372e-07,
        1.11217441e-08, -2.30409425e-06, -1.87126226e-06, -1.95006532e-06,
       -2.11627627e-06, -2.61097489e-06, -2.47727065e-06, -2.97891807e-06,
       -2.58802369e-06, -4.17562333e-06,  5.37119616e-07,  7.5